In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
import google.generativeai as genai
genai.configure(api_key=os.getenv('GCP_API_KEY'))

In [2]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory, 
    HarmBlockThreshold,
)

In [3]:
def print_responses(responses):
    for response in responses:
        for candidate in response.candidates:
            # Print the text content
            for part in candidate.content.parts:
                print(part.text)

            # Print the safety ratings
            print("Safety Ratings:")
            for rating in candidate.safety_ratings:
                print(f"  Category: {rating.category.name}")
                print(f"  Probability: {rating.probability.name}")
            print()  # Add a line break for better readability

In [4]:
model = genai.GenerativeModel("gemini-pro")

# Set parameters to reduce variability in responses
generation_config = genai.types.GenerationConfig(
    temperature=0.1,
    top_p=0.1,
    top_k=1,
    max_output_tokens=1024,
)

### Normal Showing of Text in Prompting

In [5]:
# Call Gemini API
nice_prompt = "Speak of three great things of Game of Thrones"
responses = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    for candidate in response.candidates:
        # Print the text content
        for part in candidate.content.parts:
            print(part.text)

1. **Complex and Well-Developed Characters:**

   Game of Thrones boasts
 a vast and diverse cast of characters, each with their own unique motivations, strengths, and flaws. From the scheming Cersei Lannister to the honorable Ned
 Stark, from the ruthless Tywin Lannister to the cunning Littlefinger, the series is filled with memorable and complex individuals. The show's writers took great care in developing these characters, giving them depth and nuance that made them feel real and relatable.

2. **Epic Storytelling and World-Building:**

   
Game of Thrones is set in a vast and intricate world known as Westeros, a land of seven kingdoms vying for power and control. The series masterfully weaves together multiple storylines, each following different characters and their struggles. The show's creators, David Benioff and D.B. Weiss, drew inspiration from history, mythology, and literature to create a rich and immersive world that captivated audiences.

3. **Stunning Visuals and Producti

### Showing Text while Showing Safety Ratings

In [6]:
responses = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
    stream=True,
)

print_responses(responses)


1. **Complex and Well-Developed Characters:**

   Game of Thrones boasts
Safety Ratings:
  Category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HATE_SPEECH
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HARASSMENT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_DANGEROUS_CONTENT
  Probability: NEGLIGIBLE

 a vast and diverse cast of characters, each with their own unique motivations, strengths, and flaws. From the scheming Cersei Lannister to the honorable Ned
Safety Ratings:
  Category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HATE_SPEECH
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HARASSMENT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_DANGEROUS_CONTENT
  Probability: NEGLIGIBLE

 Stark, from the ruthless Tywin Lannister to the cunning Littlefinger, the series is filled with memorable and complex individuals. The show's writers took great care in developing these characters, gi

### Looking at more challenging prompts

In [8]:
impolite_prompt = "Speak of 5 very bad things that I might say after watching Game of Thrones."

impolite_responses = model.generate_content(
    impolite_prompt,
    generation_config=generation_config,
    stream=True,
)

print_responses(impolite_responses)

1. "The ending was a complete letdown. It felt rushed and uns
Safety Ratings:
  Category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HATE_SPEECH
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HARASSMENT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_DANGEROUS_CONTENT
  Probability: NEGLIGIBLE

atisfying, like the writers didn't know how to wrap up the story properly."


2. "The character development was inconsistent. Some characters, like
Safety Ratings:
  Category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HATE_SPEECH
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HARASSMENT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_DANGEROUS_CONTENT
  Probability: NEGLIGIBLE

 Daenerys, had drastic personality changes that didn't make sense, while others, like Jon Snow, remained stagnant and underdeveloped."


3. "The plot armor was too thick. Main characters survived situations that should hav

### Continuing to look at more challenging prompts

In [9]:
rude_prompt = "Write a list of 5 terrible and very rude things I might say after I watch Game of Thrones."

rude_responses = model.generate_content(
    rude_prompt,
    generation_config=generation_config,
    stream=True,
)

print_responses(rude_responses)

1. "This show is so overrated. It's just a bunch of
Safety Ratings:
  Category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HATE_SPEECH
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HARASSMENT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_DANGEROUS_CONTENT
  Probability: NEGLIGIBLE

 violence and nudity with no real substance."
2. "The characters are all so one-dimensional and boring. I don't care about any of them
Safety Ratings:
  Category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  Probability: LOW
  Category: HARM_CATEGORY_HATE_SPEECH
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_HARASSMENT
  Probability: NEGLIGIBLE
  Category: HARM_CATEGORY_DANGEROUS_CONTENT
  Probability: NEGLIGIBLE

."
3. "The plot is so predictable and full of plot holes. I could have written a better story myself."
4. "The acting is terrible. Everyone is just overacting and hamming it up."
5. "I'm so glad this show is finally over. It was a waste
Safety Ratings:

### Setting safety thresholds

In [10]:
for value in HarmBlockThreshold:
    print(value)

HarmBlockThreshold.HARM_BLOCK_THRESHOLD_UNSPECIFIED
HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
HarmBlockThreshold.BLOCK_ONLY_HIGH
HarmBlockThreshold.BLOCK_NONE


In [11]:
safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }